In [ ]:
!pip install snscrape

#import the required libraries
import snscrape.modules.twitter as sntwitter
import pandas as pd
import pymongo
import datetime
import json

#establish API
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["twitter_scraping_db"]

# get the #tag word, date of search from user
x = str(input('Enter the #tag word you want to search for: '))
date_entry = input('Enter a date in YYYY-MM-DD format: ')
year, month, day = map(int, date_entry.split('-'))
date1 = datetime.date(year, month, day)

# Get the tweet limit and no. of days to search for the tweet
limit_days = int(input('Enter the number of days to limit: '))
tweet_limit = int(input('Enter the tweet limit: '))

end_date = date1
start_date = end_date - datetime.timedelta(limit_days)

tweet_list = []
tweet_list.clear()

#functional blog for collecting the twitter data
def keyword():
    
    query = '{} since:{} until:{}'.format(x, start_date, end_date)
    print(query)
    
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper('{}'.format(query)).get_items()):
        
        if i > tweet_limit:
            break
            
        tweet_list.append({
            'Keyword': '{}'.format(x),
            'CurrentTimeStamp': datetime.datetime.now(),
            'Tweet_Date' : tweet.date, 
            'Id': tweet.id, 
            'URL': tweet.url,
            'content': tweet.content,
            'username': tweet.user.username,
            'reply_count': tweet.replyCount,
            'retweet_count': tweet.retweetCount, 
            'language': tweet.lang,
            'source': tweet.source,
            'like_count': tweet.likeCount
            }
        )
        
keyword()
df = pd.DataFrame(tweet_list)
df.to_csv('{}.csv'.format(x))

collection_1 = mydb['Collection_1']
collection_1.insert_many(tweet_list)

with open('twitter_data_{}.json'.format(x), 'w') as f:
    s=json.dumps(tweet_list, default=str)

df